In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [3]:
tf.__version__

'1.14.0'

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv1D, Dense, Dropout, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.metrics import top_k_categorical_accuracy
def top_3_accuracy(x, y): return top_k_categorical_accuracy(x, y, 3)

In [11]:
# Constants
BATCH_SIZE = 256
EPOCHS = 10
LABELS = [["SFGPUCA-----", "Friendly Armored"], ["SFGPUCF-----", "Friendly Artillery"], ["SFGPUCI-----", "Friendly Infantry"], 
          ["SFGPUCR-----", "Friendly Reconnaisance"], ["SHGPUCA-----", "Hostile Armored"], ["SHGPUCF-----", "Hostile Artillery"], 
          ["SHGPUCI-----", "Hostile Infantry"], ["SHGPUCR-----", "Hostile Reconnaisance"]]

for label in LABELS:
    print(label[1])

Friendly Armored
Friendly Artillery
Friendly Infantry
Friendly Reconnaisance
Hostile Armored
Hostile Artillery
Hostile Infantry
Hostile Reconnaisance


In [12]:
# Data extraction and visulization
df = pd.read_csv("symbolData/SymbolData.csv")
df.head()

,x,y,t,pixels,label,id
0,"684,658,632,616,585,548,514,483,460,446,438,43...","377,338,299,277,233,183,134,88,48,24,10,2,0,2,...","42,80,94,103,119,136,154,169,187,202,221,237,2...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",Hostile Armored,SHGPUCA-----
1,"696,695,693,683,671,646,606,564,522,481,440,40...","400,400,400,396,384,356,310,253,192,140,93,58,...","14,22,37,55,71,88,105,120,138,155,170,188,204,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",Hostile Armored,SHGPUCA-----
2,"1172,1170,1167,1161,1155,1147,1147,1147,1145,1...","404,392,368,337,294,229,166,103,56,28,8,4,3,3,...","16,33,50,67,83,99,116,133,151,166,183,200,335,...","255,214,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",Friendly Artillery,SFGPUCF-----
3,"728,726,721,710,690,658,624,584,528,496,461,43...","434,432,424,411,385,346,299,239,159,113,75,42,...","14,25,41,58,74,91,109,125,142,158,174,191,209,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",Hostile Reconnaisance,SHGPUCR-----
4,"136,130,129,124,121,118,110,106,98,94,85,82,79...","9,14,15,19,22,23,29,32,37,40,45,48,51,60,62,64...","45,60,72,74,82,90,98,106,114,122,130,138,147,1...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",Hostile Infantry,SHGPUCI-----


In [7]:
df['x'] = np.array(df.x.apply(lambda x: x.split(',')))
df['y'] = np.array(df.y.apply(lambda x: x.split(',')))
df['t'] = np.array(df.t.apply(lambda x: x.split(',')))
df['label'] = df.label.apply(lambda x: x.split(','))

del df['pixels']

df.head()

,x,y,t,label
0,"[133, 128, 126, 122, 120, 114, 110, 101, 97, 8...","[26, 30, 33, 36, 38, 43, 47, 55, 59, 70, 75, 8...","[9, 15, 23, 31, 39, 47, 55, 63, 71, 79, 95, 95...","[Hostile, Armored]"
1,"[174, 171, 170, 168, 161, 158, 146, 142, 136, ...","[0, 1, 2, 3, 8, 11, 16, 20, 23, 37, 45, 49, 64...","[5, 13, 21, 29, 37, 45, 53, 61, 69, 77, 85, 93...","[Hostile, Armored]"
2,"[172, 171, 169, 163, 159, 155, 143, 137, 121, ...","[12, 13, 15, 17, 21, 23, 29, 35, 44, 49, 58, 6...","[15, 23, 31, 39, 47, 55, 71, 71, 79, 87, 95, 1...","[Hostile, Infantry]"
3,"[214, 213, 212, 208, 205, 196, 193, 188, 175, ...","[0, 0, 0, 2, 5, 10, 12, 16, 24, 29, 44, 48, 61...","[15, 23, 31, 39, 47, 55, 63, 71, 84, 87, 95, 1...","[Hostile, Armored]"
4,"[224, 222, 215, 206, 202, 196, 177, 168, 145, ...","[29, 30, 33, 39, 41, 44, 54, 60, 74, 77, 90, 9...","[6, 14, 34, 38, 46, 54, 62, 70, 78, 86, 94, 10...","[Hostile, Infantry]"


In [8]:
# Converting to ints
for i, array in enumerate(df['x']):
    for j, val in enumerate(array):
        df['x'][i][j] = int(val)

for i, array in enumerate(df['y']):
    for j, val in enumerate(array):
        df['y'][i][j] = int(val)

for i, array in enumerate(df['t']):
    for j, val in enumerate(array):
        df['t'][i][j] = int(val)

        
# Normalizing x and y
for i, array in enumerate(df['x']):
    for j, val in enumerate(array):
        df['x'][i][j] = df['x'][i][j] / max(df['x'][i])

for i, array in enumerate(df['y']):
    for j, val in enumerate(array):
        df['y'][i][j] = df['y'][i][j] / max(df['y'][i])

        
df['strokes'] = df['label']


# devide into strokes
MAX_STROKES = 0
strokes_num = 0
for i, array in enumerate(df['t']):
    new_x = []
    new_y = []
    new_t = []
    start_index_holder = 0
    for j, value in enumerate(array):
        if j != 0 and value < df['t'][i][j-1]:
            new_x.append(df['x'][i][start_index_holder:j])
            new_y.append(df['y'][i][start_index_holder:j])
            new_t.append(df['t'][i][start_index_holder:j])
            
            start_index_holder = j
    
    new_x.append(df['x'][i][start_index_holder:])
    new_y.append(df['y'][i][start_index_holder:])
    new_t.append(df['t'][i][start_index_holder:])
    
    strokes_num = len(new_x)
    if strokes_num > MAX_STROKES:
        MAX_STROKES = strokes_num
    
    df['x'][i] = np.array(new_x)
    df['y'][i] = np.array(new_y)
    df['t'][i] = np.array(new_t)
    df['strokes'][i] = strokes_num

    
# Normalizing t
for i, array in enumerate(df['t']):
    for j, subarray in enumerate(array):
        for k, val in enumerate(subarray):
            df['t'][i][j][k] = (df['t'][i][j][k] - min(df['t'][i][j])) / max(df['t'][i][j])

df.head()

,x,y,t,label,strokes
0,"[[0.37047353760445684, 0.3565459610027855, 0.3...","[[0.08904109589041095, 0.10273972602739725, 0....","[[0.0, 0.007187350263536177, 0.011020603737422...","[Hostile, Armored]",2
1,"[[0.45549738219895286, 0.4476439790575916, 0.4...","[[0.0, 0.003278688524590164, 0.006557377049180...","[[0.0, 0.0053696819496076, 0.00867410161090458...","[Hostile, Armored]",2
2,"[[0.4246913580246914, 0.4222222222222222, 0.41...","[[0.039473684210526314, 0.04276315789473684, 0...","[[0.0, 0.0099524015577672, 0.01341410644742535...","[Hostile, Infantry]",3
3,"[[0.5363408521303258, 0.5338345864661654, 0.53...","[[0.0, 0.0, 0.0, 0.0058309037900874635, 0.0145...","[[0.0, 0.013317892298784018, 0.017950202663578...","[Hostile, Armored]",2
4,"[[0.5221445221445221, 0.5174825174825175, 0.50...","[[0.08454810495626822, 0.08746355685131195, 0....","[[0.0, 0.008187134502923977, 0.019883040935672...","[Hostile, Infantry]",3


In [85]:
df['target'] = df['label']

# Converting labels into uniqe numbers
for i, label in enumerate(df['label']):
    num_label = FUNCTION_IDS.index(label[1])
    if label[0] == "Hostile":
        num_label += len(FUNCTION_IDS)
    df['target'][i] = num_label

"""
# Create target vectors as a column in the dataframe
def targeting(target):
    array = np.zeros(len(FUNCTION_IDS) * len(AFFILIATIONS), dtype = int)
    array[target] = 1
    return array
    
df['target'] = list(map(targeting, df['label']))


# Reshape number of strokes into vectors
for i, num in enumerate(df['strokes']):
    array = np.zeros(MAX_STROKES+1, dtype = int)
    array[num] = 1
    df['strokes'][i] = array
    
"""


df.head()

,x,y,t,label,strokes,target
0,"[[0.37047353760445684, 0.3565459610027855, 0.3...","[[0.08904109589041095, 0.10273972602739725, 0....","[[0.0, 0.007187350263536177, 0.011020603737422...","[Hostile, Armored]",2,2
1,"[[0.45549738219895286, 0.4476439790575916, 0.4...","[[0.0, 0.003278688524590164, 0.006557377049180...","[[0.0, 0.0053696819496076, 0.00867410161090458...","[Hostile, Armored]",2,2
2,"[[0.4246913580246914, 0.4222222222222222, 0.41...","[[0.039473684210526314, 0.04276315789473684, 0...","[[0.0, 0.0099524015577672, 0.01341410644742535...","[Hostile, Infantry]",3,3
3,"[[0.5363408521303258, 0.5338345864661654, 0.53...","[[0.0, 0.0, 0.0, 0.0058309037900874635, 0.0145...","[[0.0, 0.013317892298784018, 0.017950202663578...","[Hostile, Armored]",2,2
4,"[[0.5221445221445221, 0.5174825174825175, 0.50...","[[0.08454810495626822, 0.08746355685131195, 0....","[[0.0, 0.008187134502923977, 0.019883040935672...","[Hostile, Infantry]",3,3


In [86]:
del df['label']
df.head()

,x,y,t,strokes,target
0,"[[0.37047353760445684, 0.3565459610027855, 0.3...","[[0.08904109589041095, 0.10273972602739725, 0....","[[0.0, 0.007187350263536177, 0.011020603737422...",2,2
1,"[[0.45549738219895286, 0.4476439790575916, 0.4...","[[0.0, 0.003278688524590164, 0.006557377049180...","[[0.0, 0.0053696819496076, 0.00867410161090458...",2,2
2,"[[0.4246913580246914, 0.4222222222222222, 0.41...","[[0.039473684210526314, 0.04276315789473684, 0...","[[0.0, 0.0099524015577672, 0.01341410644742535...",3,3
3,"[[0.5363408521303258, 0.5338345864661654, 0.53...","[[0.0, 0.0, 0.0, 0.0058309037900874635, 0.0145...","[[0.0, 0.013317892298784018, 0.017950202663578...",2,2
4,"[[0.5221445221445221, 0.5174825174825175, 0.50...","[[0.08454810495626822, 0.08746355685131195, 0....","[[0.0, 0.008187134502923977, 0.019883040935672...",3,3


In [87]:
print(len(df['target']))
df.dropna(inplace=True)
print(len(df['target']))

26
26


In [ ]:
# Data normelization and train/test creation
def create_input(dataframe):
    y = [vector for vector in dataframe['target']]
    
    
    
    

df.head()

In [ ]:
train_X, train_y = np.array([[0, 0], [0, 0]]), 0
test_X, test_y = 0, 0

In [ ]:
class Model:
    def __init__(self):
        self.model = self.create_model()
        self.model.summery()
        
    def create_model(self):
        model = Sequential()
        
        model.add(BatchNormalization(input_shape = (None,)+train_X.shape[2:]))
        model.add(Conv1D(48, (5,), activation = 'relu'))
        model.add(Dropout(0.3))
        model.add(Conv1D(64, (5,), activation = 'relu'))
        model.add(Dropout(0.3))
        model.add(Conv1D(96, (3,), activation = 'relu'))
        model.add(Dropout(0.3))
        model.add(LSTM(128, return_sequences = True))
        model.add(Dropout(0.3))
        model.add(LSTM(128, return_sequences = False))
        model.add(Dropout(0.3))
        model.add(Dense(512, activation = 'relu'))
        model.add(Dropout(0.3))
        model.add(Dense(len(df['target'][0]), activation = 'softmax'))
        model.compile(optimizer = 'adam', 
                              loss = 'categorical_crossentropy', 
                              metrics = ['categorical_accuracy', top_3_accuracy])
        
        return model
    
    def train_model(self):
        self.model.fit(train_X, train_y, 
                       validation_data = (test_X, test_y), 
                       batch_size = BATCH_SIZE, 
                       epochs = EPOCHS, 
                       callbacks = callbacks_list)

In [ ]:
model = Model()